# Imaging Lab 2: Single Pixel Scanning

### EECS 16A: Designing Information Devices and Systems I, Fall 2021
<!--- Lily Bhattacharjee (lbhattacharjee@berkeley.edu)
      Wahid Rahman (wahid.rahman@berkeley.edu)
      Leyla Kabuli (lakabuli@berkeley.edu)
      Raghav Gupta (raghav.tech13@berkeley.edu)
---->

# Table of Contents

* [Instructions](#instructions)
* [Lab Policies](#policies)
* [Overview](#overview)
* [Task 1: Images, Vectors, and Matrices](#images)
    * [Task 1a: Working with Images](#task2a)
    * [Task 1b: Scanning Mask Matrix](#task2b)
* [Task 2: Imaging Real Pictures](#task3)
* [Checkoff](#checkoff)

<a id='instructions'></a>
## Instructions

* **Gently disinfect all commonly touched surfaces** at your lab station using the provided cleaning wipes.

* Complete this lab by filling in all of the required sections, marked with `"YOUR CODE HERE"` or `"YOUR COMMENTS HERE"`.


* When you finish, submit a checkoff request to get checked off for this lab. Be ready to answer a few questions to show your understanding of each section.


* Labs will be graded based on completion for teams of 2 students.

<a id='policies'></a>
## Lab Policies
* **YOU MUST ATTEND THE LAB SECTION YOU ARE ENROLLED IN. If you anticipate missing a section, please notify your GSI in advance.**
* **You are required to return all parts checked out at the beginning of the lab section unless told otherwise.**
* **You are free to stay for the full allotted time and hack around with the lab equipment, but please reserve the GSI's time for lab-related questions.**
* **Food and drinks are not allowed in the lab.** 
* **Clean up, turn off all equipment, and log off of computers before leaving.**

# <a id='overview'><span style='color:blue'>Overview</span></a>
<center>
<img src="images/system_diagram.svg" style="height:256px" />
</center>
This week, you will scan a real-life image pixel-by-pixel using a projector and light sensor circuit (a.k.a. **single pixel camera**) and write code in your Jupyter notebook to display the captured image. 

You will begin by checking that the circuit you built last time (with minor modifications) works and that the projector is properly connected to the computer. 

Next, you will write code to generate the "mask" patterns that the projector uses to scan the object. 

Then, you will use your generated mask matrix to photograph an image with the projector and ambient light sensor setup (seen below). Finally, you will write code to recreate the image from light sensor readings that are sent to your computer via the Launchpad.

To prevent room lighting from affecting these measurements, the projector setup is usually placed in a closed cardboard box. 

<br><br>
<center>
**Cory 140 Setup**
<img src="images/side.png" style="height:350px" />
<br>
**Cory 125 Setup**
<img src="images/125_setup.jpg" style="height:350px" />
</center>

The single pixel imaging process would involve the following steps:
<ul>
    <li> The projector illuminates the object with a mask. </li>
    <li> The ambient light sensor detects the total amount of light reflected off the object. More light leads to more current through the sensor. </li>
    <li> The analog circuit converts the sensor's current into an output voltage. More light $\rightarrow$ higher sensor current $\rightarrow$ higher output voltage. </li>
    <li> This analog voltage is converted into a digital brightness value. </li>
</ul>    

<b>Note:</b> In the real world, we come across random irregular fluctuations while taking measurements. This is called noise. It is important to consider noise while designing any system, and this lab is no different. You will learn more about noisy imaging in the Imaging 3 lab.
</font>    

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%run scripts/test.py

<a id='images'></a>
# <span style='color:blue'>Task 1: Images, Vectors, and Matrices </span>

<a id='task2a'></a>
## <span style="color:blue">Task 1a: Working with Images</span>

<br>
    
How can we represent an image? Consider a 5x5 grayscale image, where each of the 25 pixel intensities vary in shades of gray. One way to represent this is with a 2D matrix (2D NumPy array). The values stored in this array, varying from 0 to 1, correspond to different shades of gray: the lower the pixel value, the darker the pixel.  

For example, take the 5x5 **`gradient_image`** shown below. Starting from the top-left pixel (pixel[0,0]), each pixel becomes progressively brighter as you traverse the image row-by-row. Note that we will be using 0 indexing in lab as most programming languages (including Python) index in lists starting from 0.

<center>
<img src="images/gradient.JPG" align="center" style="height:200px" />
<figcaption>Gradient image example</figcaption>
</center>

We can create this in Python using a $5 \times 5$ NumPy 2D array called **`gradient_image`** with *linearly-spaced* floating point values from 0 to 1. The Python code to generate this is provided for you below. Take a look at the numerical 2D array and the corresponding image that is displayed by using the `imshow` function.
</font>

In [ ]:
# A 5x5 gradient image with values from 0 to 1.
gradient_image = np.linspace(0, 1, 25).reshape([5, 5])

print(gradient_image)
plt.imshow(gradient_image, cmap = "gray", interpolation = "nearest")

**<span style="color:red">What color does 1.0 correspond to? What about 0?</span>**

`YOUR COMMENTS HERE`

Instead of treating our images as 2D matrices, we can "reshape" or "flatten" our images into 1-D vectors. That is, instead of having a $5 \times 5$ matrix for our image, we can represent it using a $25 \times 1$ vector. This makes it simpler for us to use the linear algebra techniques learned in class for image processing. 
    
Let's look at the 3x3 example image below (colored for illustrative purposes). How can we transform this 2D vector matrix into a 1-D column vector? 
    
Essentially, the $0$th row is flipped on its side (rotate clockwise 90 degrees), such that its left-most element is on top and its right-most element is on the bottom. The $1$st row is also flipped on its side in the same way and appended below. These steps are repeated for each subsequent row of the original 2D image until you build a $9 \times 1$ **column vector**.    

<center>
<img src="images/matrix_to_col_new.png" style="width:500px"/>
</center>

Mathematically, each pixel value in the $3 \times 3$ image is represented as a variable $p_{ij}$, where $i$ is the row and $j$ is the column associated with the pixel location. This same image represented as a 1-D column vector (called $\vec{i}$) is:

$$\vec{i} = \begin{bmatrix} p_{00} \\ p_{01} \\ p_{02} \\ p_{10} \\ p_{11} \\ p_{12} \\ p_{20} \\ p_{21} \\ p_{22} \end{bmatrix}$$    
    
The procedure described above can be used to convert any $N \times M$ 2D image into a `num_pixels` $\times 1$ **column vector**, where `num_pixels` $= N \times M$.    

**<span style="color:red">Convert the 5x5 `gradient_image` that you created above into a 25x1 column vector `gradient_image_vector` and display it. You will find the command `np.reshape` helpful. What pattern do you notice? Think about why you see this pattern.</span>**

In [ ]:
# TODO: Convert the 5x5 matrix into a 25x1 column vector
gradient_image_vector = # YOUR CODE HERE

# Display the vector
plt.imshow(gradient_image_vector, cmap = "gray", interpolation = "nearest")
plt.xticks([])
plt.yticks(np.arange(0, 30, 5))
plt.show()

<a id='task2b'></a>
## <span style="color:blue">Task 1b: Scanning Mask Matrix</span>

Next, we will create a "mask" matrix (array) to enable our simulated projector to illuminate and scan individual pixels, one at a time. This is the magic behind our virtual single pixel camera. 

If **`gradient_image_vector`** is represented by the column vector variable $\vec{i}$, the act of transforming $\vec{i}$ by a matrix $H$ into another 1D column vector $\vec{s}$ is represented mathematically as:

$$ \vec{s} = H \vec{i} $$

This matrix multiplication represents what happens when we scan an image with our virtual single pixel camera! In the context of a real-world imaging system, $H$ represents the scanning "mask matrix," whose rows are projected one-by-one onto the image we want to scan. $\vec{s}$ represents digitized readings from the analog circuit's light sensor. In the context of the simulation, each mask in $H$ will be reshaped to match the dimensions of the image, and subsequently overlaid one at a time, with only illuminated pixel measurements contributing to the corresponding mask measurement. Each element $s_k$ of $\vec{s}$ corresponds to one scan (using one row $k$ of $H$, that we refer to as $H_k$). 

Each 1D **row of $H$** represents a **mask**. But what is a mask? In the context of our virtual single pixel camera, a mask is a way to highlight certain locations in the image while hiding others during scanning. For a 3x4 image (where 3 = height, 4 = width), a mask taken from **row 0 of $H$** is represented as the $1 \times 12$ row vector below: 

$$
H_0 
= \begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\end{bmatrix}
$$

However, the mask must first be converted into its 2D form, as shown below, before it's projected over the 2D image. The mask exposes only the top-left pixel of the 2D image and hides all other pixels. Note that you can convert a 2D mask into a 1D row of $H$ by appending each of the 2D mask's rows to the right of the previous row.
<br><br>
<center>
<img src="images/black_hite.png" style="width:400px"/>
</center>
 
To expose each pixel of the 3x4 image $\vec{i}$ individually, we would need a 12x12 $H$ that has 12 masks (rows), each with a single white "exposed" pixel in a unique location. This means that **row 1 of $H$** (exposing $iv_{01}$) would look like:

$$
H_1 
= \begin{bmatrix} 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\end{bmatrix}
$$
<br><br>
<center>
<img src="images/black_white_shifted.jpg" style="width:400px"/>
</center>

The process of masking an image, then uncovering one pixel at a time, and sensing the resultant ambient light performs the matrix multiplication $\vec{s} = H \vec{i}$ in real life. This equation implies that each element of the sensor output vector $\vec{s}$ can be determined as:
    
$$s_k = H_k \vec{i}$$

Where the $k$th sensor reading is determined by the mask given by $k$th row of $H$, $H_k$. Thus, projecting the 2D representation of $H_0$ shown above onto a 3x4 image represented by the column vector $\vec{i}$ to obtain the sensor reading $s_0$ would be mathematically equivalent to:

$$
s_0 = \begin{bmatrix} 1 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0\end{bmatrix} \vec{i}
$$

**<span style="color:red">What dimensions does the mask matrix $H$ have for a 5x5 image? Why? </span>**

`YOUR COMMENTS HERE`

**<span style="color:red">
Create the mask matrix $H$ for a 5x5 image.</span>**

*Hint: Google the function `np.eye`.*

In [ ]:
# TODO: Create the mask matrix `H` for scanning a 5x5 image (be careful about the dimensions!)
H = # YOUR CODE HERE

# Test H for correctness
test1b_H(H)

# Display this matrix
plt.imshow(H, cmap = "gray", interpolation = "nearest")

**<span style="color:red">
Multiply the $H$ matrix with `gradient_image_vector`. Remember to use `np.dot` to do matrix multiplication.</span>**

In [ ]:
# TODO: Multiply `H` and `gradient_image_vector`
s = # YOUR CODE HERE

# Display the result and compare it to `gradient_image_vector`
plt.imshow(s, cmap = "gray", interpolation = "nearest")
plt.xticks([])
plt.yticks(np.arange(0, 30, 5))
plt.show()

**<span style="color:red">Is the resultant `s` equal to `gradient_image_vector`? Why?</span>**

`YOUR COMMENTS HERE`

What happens when this matrix multiplication is performed? To reiterate, each row of $H$ is responsible for "illuminating," or selecting, a single pixel in the gradient image. `gradient_image_vector` was created by converting the 5x5 `gradient_image` into a 1D *column vector*. Similarly, *every row* in $H$ can be represented as a 5x5 image that, in real imaging, would be projected over `gradient_image`. 

**<span style="color:red">
Iterate through each row of the matrix $H$. *Reshape* each row into a 5x5 image, and check that each row illuminates a unique pixel of the original 5x5 image! Based on $ \vec{s} = H \vec{i} $, why are the rows of $H$ used for masking when $\vec{i}$ is a column vector?</span>**

In [ ]:
# Iterate through rows of matrix H and form individual masks
plt.figure(figsize = (20, 20)) 
for k in range(25):
    plt.subplot(5, 5, k + 1)
    
    mask = # YOUR CODE HERE
    
    plt.imshow(mask, cmap = "gray", interpolation = "nearest")
    plt.title("Mask " + str(k) + " = Row " + str(k) + " of Matrix H")

Each of the images above are masks. During a single scan, we project one of these masks over our image. The white pixel illuminates a particular location on the image that we want to capture, and the black pixels obscure the other parts of the image. Thus, using the rows of $H$, we gather information one pixel at a time.

Let's try to make another mask matrix, $H_{alt}$, that's a little more complicated. We want the first (upper) half of $\vec{s}$ to contain information on every other pixel of the image (i.e. $p_{00}$, $p_{02}$, $p_{20}$...). The second (lower) half of $\vec{s}$ should consist of information on the pixels that the first half skipped (i.e. $p_{01}$, $p_{03}$, $p_{10}$).

**<span style="color:red">
Generate $H_{alt}$ for a 5x5 image whose first half (in terms of rows) illuminates every other pixel of $\vec{i}$ and whose second half illuminates the pixels that were skipped. Multiply $H_{alt}$ by `gradient_image_vector` to produce the new output vector $\vec{s}_{alt}$.
</span>**

<b>Hint</b>: Try to use rows from the existing H matrix. The syntax to slice an NumPy array is as follows `array[star_row:stop_row:skip,start_col:stop_col:skip]`. `np.vstack` may be very helpful (try Googling it). If done well, the code to generate H_alt should only be 1 short line.

<b>Hint 2</b>: Here's how $H_{alt}$ would look for an image of size 4x4.
<br><br>
<center>
<img src="images/H_alt_4x4.png" style="width:300px"/>
    <figcaption>$H_{alt}$ for a 4x4 image</figcaption>
</center>

In [ ]:
# TODO: Create the new mask matrix `H_alt` for a 5x5 image
H_alt = # YOUR CODE HERE

# Test H_alt for correctness
test1b_H_alt(H_alt)

# Display `H_alt`
plt.figure()
plt.imshow(H_alt, cmap = "gray", interpolation = "nearest")

# TODO: Multiply `H_alt` and `gradient_image_vector`
s_alt = # YOUR CODE HERE

# Display the result `s` and compare to `gradient_image_vector`
plt.figure()
plt.imshow(s_alt, cmap = "gray", interpolation = "nearest")
plt.xticks([])
plt.yticks(np.arange(0, 30, 5))
plt.show()

Because of how we designed `H_alt`, `s_alt` is clearly different from `gradient_image_vector`. Each pixel of `gradient_image_vector` is still illuminated only once by `H_alt`, but the order in which the pixels are illuminated has changed. Therefore, we say that `s_alt` is a "scrambled" version of `gradient_image_vector`. How could we "reconstruct" $\vec{s}_{alt}$ back into the original `gradient_image_vector`? 

Recall that our original matrix $H$ was actually the **identity matrix** $I_n$. In other words, the original $\vec{s}$ was computed as:

$ \vec{s} = H \vec{i} = I_n \vec{i}$

<br />    
Using the alternate mask, $H_{alt}$, we compute the alternate output $\vec{s}_{alt}$ as:

$ \vec{s}_{alt} = H_{alt} \vec{i} $

To "reconstruct" $\vec{s}_{alt}$ back into the original `gradient_image_vector` (i.e. $\vec{i}$), we must find a matrix $M$ that multiplies $\vec{s}_{alt}$ to make the following true:

$ M \vec{s}_{alt} = \vec{i} $

i.e.

$ M H_{alt} \vec{i} = \vec{i} $

**<span style="color:red">What should M be to recover $\vec{i}$?</span>**

`YOUR COMMENTS HERE`

**<span style="color:red">Write code to reconstruct `gradient_image_vector` from `s_alt`.</span>**

In [ ]:
# TODO: Reconstruct `gradient_image_vector`
M = # YOUR CODE HERE
gradient_image_vector_reconstruct = # YOUR CODE HERE

# Display M
plt.figure()
plt.imshow(M, cmap = "gray", interpolation = "nearest")
plt.title("M")
plt.show()

# Display M*H_alt
plt.figure()
plt.imshow(np.dot(M,H_alt), cmap = "gray", interpolation = "nearest")
plt.title("M*H_alt")
plt.show()

# Display the result
plt.imshow(gradient_image_vector_reconstruct, cmap = "gray", interpolation = "nearest")
plt.xticks([])
plt.yticks(np.arange(0, 30, 5))
plt.show()

<a id='task3'></a>
# <span style="color:blue">Task 2: Imaging Real Pictures</span>

Finally, we will use our two matrices to simulate imaging a more complex picture. Because our picture is fairly large, we want each individual mask to have dimensions 32x32 (i.e. height = 32, width = 32). Think about how big the mask matrix was for the 5x5 example. In the next cell, you'll use the same logic to find the dimensions for the mask matrix of a 32x32 image.

**<span style="color:red">
Recreate both the $H$ and $H_{alt}$ mask matrices for 32x32 images. </span>**

In [ ]:
%run scripts/helpers.py

In [ ]:
# TODO: Recreate `H` with new dimensions
H = # YOUR CODE HERE

plt.figure(figsize = (10, 10))
plt.imshow(H, cmap = "gray", interpolation = "nearest")

In [ ]:
# TODO: Recreate `H_alt` with new dimensions      
H_alt = # YOUR CODE HERE

plt.figure(figsize = (10, 10))
plt.imshow(H_alt, cmap = "gray", interpolation = "nearest")

Note: The H_alt to match new dimensions looks somewhat different from the previous 25x25 H_alt because the pixels are too tiny to be distinguished. As long as you see a pattern similar to 25x25 H_alt and pass the following test, you're correct!

Let's make sure that the two matrices we made are invertible and the correct size. Run the code block below to invert the matrices and test dimensions - if any of the lines fail, it means the code used to generate either matrix resulted in an incorrect size or non-invertible, linearly dependent matrix, which is incorrect. 

In [ ]:
test2(H, H_alt)

Our mask matrices must be saved as files before they can be used to perform real imaging. The files are read by our imaging script, as seen below. 

**<span style="color:red">Run the cell below to save `H` and `H_alt`!</span>**

In [ ]:
np.save("saved_data/H.npy", H)
np.save("saved_data/H_Alt.npy", H_alt)

You are done for the week! Save your code and circuit for next lab, where you will illuminate multiple pixels per mask!

## **<span style="color:red"> Do not take apart your setup before being checked off. </span>** 
**Afterwards, you may take the Launchpad and circuit kit with you. Remember to bring it back next week!**

<a id='checkoff'></a>
## Checkoff
When you are ready to get checked off,
1. Fill out the checkoff google form. **[Checkoff Form](https://docs.google.com/forms/d/e/1FAIpQLSeq9w_gvQKS37hUXgTDtiYhu9gNmu5OFzglIYAF4nfXQc5mOA/viewform?usp=sf_link)**
2. Submit a **checkoff** request on the lab queue. **[Checkoff queue](https://lab.eecs16a.org)** Only one person from your group should submit the queue request, but everyone must submit their own google form. 

Your GSI or a Lab Assistant will join you when they are available and go through some checkoff questions with your group. They will go through the checkoff list in order. Please be patient!


#### Post-checkoff Clean Up: (this applies to each week's lab)
1. Pack away your launchpad and lab kit materials to bring home  - make a note to bring them back next week!
2. Throw away any trash at your station
3. Return the wires from the back walls
4. SIGN OUT of the computers - DO NOT SHUT DOWN
5. Check that other machines at your lab station are powered off